In [82]:
schema = {"openapi":"3.1.0","info":{"title":"FastAPI","version":"0.1.0"},"paths":{"/titles":{"get":{"summary":"Get Titles","operationId":"get_titles_titles_get","responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"$ref":"#/components/schemas/GetTitlesReturn"}}}}}}},"/all_story_full_paths":{"get":{"summary":"Get All Story Full Paths","operationId":"get_all_story_full_paths_all_story_full_paths_get","responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"$ref":"#/components/schemas/GetStoryFullPathsReturn"}}}}}}},"/story":{"post":{"summary":"Get Story","operationId":"get_story_story_post","requestBody":{"content":{"application/json":{"schema":{"$ref":"#/components/schemas/GetStoryInput"}}},"required":True},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"$ref":"#/components/schemas/Story"}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}},"/register_managers":{"get":{"summary":"Registor Managers","operationId":"registor_managers_register_managers_get","responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"type":"string","title":"Response Registor Managers Register Managers Get"}}}}}}},"/api-schema":{"get":{"summary":"Get Api Schema","operationId":"get_api_schema_api_schema_get","responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}}}}},"/simple":{"post":{"summary":"With Default","operationId":"with_default_simple_post","requestBody":{"content":{"application/json":{"schema":{"$ref":"#/components/schemas/SimpleProps"}}},"required":True},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"$ref":"#/components/schemas/SimpleOutProps"}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}}},"components":{"schemas":{"GetStoryFullPathsReturn":{"properties":{"story_full_paths":{"items":{"type":"string"},"type":"array","title":"Story Full Paths"}},"type":"object","required":["story_full_paths"],"title":"GetStoryFullPathsReturn"},"GetStoryInput":{"properties":{"story_full_paths":{"type":"string","title":"Story Full Paths"}},"type":"object","required":["story_full_paths"],"title":"GetStoryInput"},"GetTitlesReturn":{"properties":{"titles":{"items":{"type":"string"},"type":"array","title":"Titles"}},"type":"object","required":["titles"],"title":"GetTitlesReturn"},"HTTPValidationError":{"properties":{"detail":{"items":{"$ref":"#/components/schemas/ValidationError"},"type":"array","title":"Detail"}},"type":"object","title":"HTTPValidationError"},"SimpleOutProps":{"properties":{"output":{"type":"integer","title":"Output"}},"type":"object","required":["output"],"title":"SimpleOutProps"},"SimpleProps":{"properties":{"arg1":{"type":"integer","title":"Arg1","default":1},"arg2":{"type":"integer","title":"Arg2","default":2}},"type":"object","title":"SimpleProps"},"Story":{"properties":{"name":{"type":"string","title":"Name"},"meta":{"anyOf":[{"type":"object"},{"type":"null"}],"title":"Meta"},"parent":{"type":"string","title":"Parent"},"full_path":{"type":"string","title":"Full Path"},"kwargs":{"anyOf":[{"type":"object"},{"type":"null"}],"title":"Kwargs"},"docs":{"anyOf":[{"type":"string"},{"type":"null"}],"title":"Docs"},"source":{"anyOf":[{"type":"string"},{"type":"null"}],"title":"Source"},"typehints":{"anyOf":[{"additionalProperties":{"type":"string"},"type":"object"},{"type":"null"}],"title":"Typehints"}},"type":"object","required":["name","meta","parent","full_path","kwargs","docs","source","typehints"],"title":"Story"},"ValidationError":{"properties":{"loc":{"items":{"anyOf":[{"type":"string"},{"type":"integer"}]},"type":"array","title":"Location"},"msg":{"type":"string","title":"Message"},"type":{"type":"string","title":"Error Type"}},"type":"object","required":["loc","msg","type"],"title":"ValidationError"}}}}

In [ ]:
schema

In [ ]:
from python_storybook.core import StoryManager

In [83]:
from pydantic import BaseModel, create_model
from typing import Optional

import inflection

snake_case = "get_titles_titles_get"
camel_case = inflection.camelize(snake_case, False)  # 첫 글자를 소문자로 유지
print(camel_case, inflection.camelize(snake_case, uppercase_first_letter=True))



getTitlesTitlesGet GetTitlesTitlesGet


In [ ]:
paths = schema['paths']

In [ ]:
paths['/story']['post']

In [ ]:
[ 
    paths['/story']['post']['operationId'],
    paths['/story']['post']['requestBody']['content']['application/json']['schema']['$ref'].split('/')[-1],
    paths['/story']['post']['responses']['200']['content']['application/json']['schema']['$ref'].split('/')[-1]
]

In [ ]:
class FunctionDefinition(BaseModel):
    func_name:str
    input_props_name:Optional[str]
    output_props_name:Optional[str]

In [ ]:
def extract_function_definition(json_data, path, request_type):
    func_name = json_data['paths'][path][request_type]['operationId']
    
    try:
        ref_request_body = json_data['paths'][path][request_type]['requestBody']['content']['application/json']['schema']['$ref']
        input_props_name = ref_request_body.split('/')[-1]
    except:
        input_props_name = None
    
    try:
        ref_response_body = json_data['paths'][path][request_type]['responses']['200']['content']['application/json']['schema']['$ref']
        output_props_name = ref_response_body.split('/')[-1]
    except:
        output_props_name = None
    
    
    
    return FunctionDefinition(
        func_name=inflection.camelize(func_name, False),
        input_props_name=input_props_name,
        output_props_name=output_props_name,
    )

In [ ]:
extract_function_definition(schema, '/story', 'post')

In [ ]:
extract_function_definition(schema, '/titles', 'get')

In [84]:
route_names = list(schema['paths'].keys())
route_names

['/titles',
 '/all_story_full_paths',
 '/story',
 '/register_managers',
 '/api-schema',
 '/simple']

In [85]:
definitions = []

for route_name in route_names:
    for route_type in ['get', 'post']:
        try:
            definitions.append(extract_function_definition(schema, route_name, route_type))
        except:
            pass


```tsx
import React, { useState} from 'react'
import { DefaultApi, Configuration, GetStoryInput, Story } from './ts-client';  // Adjust the path as necessary

const apiConfig = new Configuration({ basePath: "http://localhost:8080" });
const titlesApi = new DefaultApi(apiConfig);

export const defaultInput:GetStoryInput = {
    story_full_paths:'Example/add'
}

export const GetStoryStoryPost:React.FC<GetStoryInput> = (
    story_full_paths
) => {
    const [response, setResponse] = useState<Story|undefined>()

    const handleClick = async () => {
        const response = await titlesApi.getStoryStoryPost(story_full_paths)
        setResponse(response.data)
    } 
    
    return <div>
                <button onClick={handleClick}>Execute</button>
                <p>{JSON.stringify(response, null, 2)}</p>   
            </div>
}

export default GetStoryStoryPost

```

In [86]:
component_template = r'''
import React, { useState} from 'react'
import { DefaultApi, Configuration, \[input_names\], \[output_names\] } from './ts-client';  // Adjust the path as necessary

const apiConfig = new Configuration({ basePath: "\[base_url\]" });
const titlesApi = new DefaultApi(apiConfig);

export const defaultInput:\[input_names\] = \[kwarg\]

export const \[Func_name\]:React.FC<\[input_names\]> = (
    inputProps
) => {
    const [response, setResponse] = useState<\[output_names\]|undefined>()

    const handleClick = async () => {
        const response = await titlesApi.\[func_name\](inputProps)
        setResponse(response.data)
    } 
    
    return <div>
                <button onClick={handleClick}>Execute</button>
                <p>{JSON.stringify(response, null, 2)}</p>   
            </div>
}

export default \[Func_name\]
'''

In [87]:
edit1 = component_template.replace(r'{', r'{{')
edit2 = edit1.replace(r'}', r'}}')
edit3 = edit2.replace(r'\[', r'{')
edit4 = edit3.replace(r'\]', r'}')

In [88]:
edit4

'\nimport React, {{ useState}} from \'react\'\nimport {{ DefaultApi, Configuration, {input_names}, {output_names} }} from \'./ts-client\';  // Adjust the path as necessary\n\nconst apiConfig = new Configuration({{ basePath: "{base_url}" }});\nconst titlesApi = new DefaultApi(apiConfig);\n\nexport const defaultInput:{input_names} = {kwarg}\n\nexport const {Func_name}:React.FC<{input_names}> = (\n    inputProps\n) => {{\n    const [response, setResponse] = useState<{output_names}|undefined>()\n\n    const handleClick = async () => {{\n        const response = await titlesApi.{func_name}(inputProps)\n        setResponse(response.data)\n    }} \n    \n    return <div>\n                <button onClick={{handleClick}}>Execute</button>\n                <p>{{JSON.stringify(response, null, 2)}}</p>   \n            </div>\n}}\n\nexport default {Func_name}\n'

In [89]:
template = '''
import React, {{ useState}} from 'react'
import {{ DefaultApi, Configuration, {input_names}, {output_names} }} from './ts-client';  // Adjust the path as necessary

const apiConfig = new Configuration({{ basePath: "{base_url}" }});
const titlesApi = new DefaultApi(apiConfig);

export const defaultInput:GetStoryInput = {kwarg}

export const {Func_name}:React.FC<> = (
    inputProps
) => {{
    const [response, setResponse] = useState<{output_names}|undefined>()

    const handleClick = async () => {{
        const response = await titlesApi.{func_name}(inputProps)
        setResponse(response.data)
    }} 
    
    return <div>
                <button onClick={{handleClick}}>Execute</button>
                <p>{{JSON.stringify(response, null, 2)}}</p>   
            </div>
}}

export default {Func_name}
'''

In [90]:
definitions

[FunctionDefinition(func_name='getTitlesTitlesGet', input_props_name=None, output_props_name='GetTitlesReturn'),
 FunctionDefinition(func_name='getAllStoryFullPathsAllStoryFullPathsGet', input_props_name=None, output_props_name='GetStoryFullPathsReturn'),
 FunctionDefinition(func_name='getStoryStoryPost', input_props_name='GetStoryInput', output_props_name='Story'),
 FunctionDefinition(func_name='registorManagersRegisterManagersGet', input_props_name=None, output_props_name=None),
 FunctionDefinition(func_name='getApiSchemaApiSchemaGet', input_props_name=None, output_props_name=None),
 FunctionDefinition(func_name='withDefaultSimplePost', input_props_name='SimpleProps', output_props_name='SimpleOutProps')]

In [91]:
definition:FunctionDefinition = definitions[-1]

In [92]:
formatted = edit4.format(
    func_name=definition.func_name,
    Func_name=definition.func_name,
    input_names=definition.input_props_name,
    output_names=definition.output_props_name,
    base_url="http://localhost:8080",
    kwarg=r"{arg1:1, arg2:2}"
)

In [93]:
print(formatted)


import React, { useState} from 'react'
import { DefaultApi, Configuration, SimpleProps, SimpleOutProps } from './ts-client';  // Adjust the path as necessary

const apiConfig = new Configuration({ basePath: "http://localhost:8080" });
const titlesApi = new DefaultApi(apiConfig);

export const defaultInput:SimpleProps = {arg1:1, arg2:2}

export const withDefaultSimplePost:React.FC<SimpleProps> = (
    inputProps
) => {
    const [response, setResponse] = useState<SimpleOutProps|undefined>()

    const handleClick = async () => {
        const response = await titlesApi.withDefaultSimplePost(inputProps)
        setResponse(response.data)
    } 
    
    return <div>
                <button onClick={handleClick}>Execute</button>
                <p>{JSON.stringify(response, null, 2)}</p>   
            </div>
}

export default withDefaultSimplePost

